# Introduction
This notebook uses my trained model generated from https://www.kaggle.com/hdsk38/pytorch-starter-train-efficientnet to predict the hidden test dataset. Please go to the notebook to see the details of trained model. Many codes are forked from https://www.kaggle.com/rhtsingh/pytorch-training-inference-efficientnet-baseline. Thank you [
@rhtsingh](https://www.kaggle.com/rhtsingh)!

I hope this notebook will help more kagglers to join this competition. Cheers! :)

In [ ]:
!pip install ../input/efficientnet-pytorch/EfficientNet-PyTorch/EfficientNet-PyTorch-master/ > /dev/null # no output
!pip install torch_optimizer --no-index --find-links=file:///kaggle/input/torch-optimizer/torch_optimizer

In [ ]:
import os
import gc
gc.enable()
import sys
import math
import json
import time
import random
from glob import glob
from datetime import datetime

import cv2
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
from PIL import Image
import multiprocessing
from sklearn.preprocessing import LabelEncoder

import torch
import torchvision
from torch import Tensor
from torchvision import transforms
import torch.nn as nn
import torch.nn.functional as F
from torch.autograd import Variable
from torch.nn.parameter import Parameter
from torch.optim import lr_scheduler
from torch.utils.data import DataLoader, Dataset
from torch.utils.data.sampler import SequentialSampler
from tqdm import tqdm

import efficientnet_pytorch

import torch_optimizer as optim
import albumentations as A

import sklearn

import warnings
warnings.filterwarnings("ignore")

In [ ]:
IN_KERNEL = os.environ.get('KAGGLE_WORKING_DIR') is not None
BATCH_SIZE = 64
NUM_WORKERS = multiprocessing.cpu_count()
NUM_TOP_PREDICTS = 1

In [ ]:
test_dir = '../input/landmark-recognition-2021/test/'

In [ ]:
class ImageDataset(torch.utils.data.Dataset):
    def __init__(self, dataframe: pd.DataFrame, image_dir:str, mode: str):
        self.df = dataframe
        self.mode = mode
        self.image_dir = image_dir
        
        transforms_list = []
        if self.mode == 'train':
            # Increase image size from (64,64) to higher resolution,
            # Make sure to change in RandomResizedCrop as well.
            transforms_list = [
                transforms.Resize((64,64)),
                transforms.RandomHorizontalFlip(),
                transforms.RandomChoice([
                    transforms.RandomResizedCrop(64),
                    transforms.ColorJitter(0.2, 0.2, 0.2, 0.2),
                    transforms.RandomAffine(degrees=15, translate=(0.2, 0.2),
                                            scale=(0.8, 1.2), shear=15,
                                            resample=Image.BILINEAR)
                ]),
                transforms.ToTensor(),
                transforms.Normalize(mean=[0.485, 0.456, 0.406],
                                      std=[0.229, 0.224, 0.225]),
            ]
        else:
            transforms_list.extend([
                # Keep this resize same as train
                transforms.Resize((64,64)),
                transforms.ToTensor(),
                transforms.Normalize(mean=[0.485, 0.456, 0.406],
                                      std=[0.229, 0.224, 0.225]),
            ])
        self.transforms = transforms.Compose(transforms_list)

    def __getitem__(self, index: int):
        image_id = self.df.iloc[index].id
        image_path = f"{self.image_dir}/{image_id[0]}/{image_id[1]}/{image_id[2]}/{image_id}.jpg"
        image = Image.open(image_path)
        image = self.transforms(image)

        if self.mode == 'test':
            return {'image':image}
        else:
            return {'image':image, 
                    'target':self.df.iloc[index].landmark_id}

    def __len__(self) -> int:
        return self.df.shape[0]

In [ ]:
class EfficientNetEncoderHead(nn.Module):
    def __init__(self, depth, num_classes):
        super(EfficientNetEncoderHead, self).__init__()
        self.depth = depth
        self.base = efficientnet_pytorch.EfficientNet.from_name(f'efficientnet-b{self.depth}')#tropicbird
        pretrained_file = glob(f'../input/efficientnet-pytorch/efficientnet-b{self.depth}*')[0]
        checkpoint = torch.load(pretrained_file)
        self.base.load_state_dict(checkpoint)
        
        self.avg_pool = nn.AdaptiveAvgPool2d(1)
        self.output_filter = self.base._fc.in_features
        self.classifier = nn.Linear(self.output_filter, num_classes)
    def forward(self, x):
        x = self.base.extract_features(x)
        x = self.avg_pool(x).squeeze(-1).squeeze(-1)
        x = self.classifier(x)
        return x

In [ ]:
def inference(data_loader, model):
    model.eval()

    activation = nn.Softmax(dim=1)
    all_predicts, all_confs, all_targets = [], [], []

    with torch.no_grad():
        for i, data in enumerate(tqdm(data_loader, disable=IN_KERNEL)):
            if data_loader.dataset.mode != 'test':
                input_, target = data['image'], data['target']
            else:
                input_, target = data['image'], None

            output = model(input_.cuda())
            output = activation(output)

            confs, predicts = torch.topk(output, NUM_TOP_PREDICTS)
            all_confs.append(confs)
            all_predicts.append(predicts)

            if target is not None:
                all_targets.append(target)

    predicts = torch.cat(all_predicts)
    confs = torch.cat(all_confs)
    targets = torch.cat(all_targets) if len(all_targets) else None

    return predicts, confs, targets

In [ ]:
def generate_submission(test_loader, model, label_encoder):
    predicts_gpu, confs_gpu, _ = inference(test_loader, model)
    predicts, confs = predicts_gpu.cpu().numpy(), confs_gpu.cpu().numpy()

    #The modified labels are inversed to the original labels
    labels = [label_encoder.inverse_transform(pred) for pred in predicts]
    print('labels')
    print(np.array(labels))
    print('confs')
    print(np.array(confs))

    sub = test_loader.dataset.df

    def concat(label: np.ndarray, conf: np.ndarray):
        return ' '.join([f'{str(L)} {str(np.round(c,4))}' for L, c in zip(label, conf)])
    sub['landmarks'] = [concat(label, conf) for label, conf in zip(labels, confs)]

    sub = sub.set_index('id')
    sub.to_csv('submission.csv')

In [ ]:
test_filenames=[]
for dirname, _, filenames in os.walk('../input/landmark-recognition-2021/test'):
    for filename in filenames:
        test_filenames.append(filename.split(".")[0])
test=pd.DataFrame({"id":test_filenames,"landmarks":""})

In [ ]:
test_dataset = ImageDataset(test, test_dir, mode='test')
test_loader = DataLoader(test_dataset, 
                         batch_size=BATCH_SIZE,
                         shuffle=False, num_workers=NUM_WORKERS)

# Load label encoder

In [ ]:
label_encoder = LabelEncoder()
label_encoder.classes_ = np.load('../input/pytorch-starter-train-efficientnet/classes.npy')

# Load model weights

In [ ]:
model = EfficientNetEncoderHead(depth=0, num_classes=len(label_encoder.classes_))
model.cuda()
model.load_state_dict(torch.load("../input/pytorch-starter-train-efficientnet/new_weight_efficientnet.pth"))

# Make prediction

In [ ]:
generate_submission(test_loader, model, label_encoder)
pd.read_csv("./submission.csv")